# This notebook

* Generates .turb setup files for Athena++ to run on Freya
* * Generates .sh files for running the code

# Setup

In [137]:
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pickle


import sys
import os
sys.path.append(os.path.abspath('/freya/ptmp/mpa/wuze/codes'))
from jason import plotting_def, plot_prettier

plotting_def()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Params

## Functions

In [138]:
def _into_dict(start_key, end_key):
    all_keys = list(globals().keys())
    dict_keys = all_keys[all_keys.index(start_key) : all_keys.index(end_key)+1]
    parse_dict = {}
    for key in dict_keys:
        parse_dict[key] = globals()[key]
    return parse_dict

In [139]:
"""Define constants"""

class unit():
    def __init__(self):        
        # length, time, and mass constants
        self.CONST_pc  = 3.086e18
        self.CONST_yr  = 3.154e7
        self.CONST_amu = 1.66053886e-24
        self.CONST_kB  = 1.3806505e-16
        self.unit_length = self.CONST_pc*1e3  # 1 kpc
        self.unit_time   = self.CONST_yr*1e6  # 1 Myr
        self.unit_density = self.CONST_amu    # 1 mp/cm-3
        self.unit_velocity = self.unit_length/self.unit_time
        self.KELVIN = self.unit_velocity*self.unit_velocity*self.CONST_amu/self.CONST_kB
        self.unit_q = (self.unit_density * (self.unit_velocity**3))/self.unit_length
        self.g = 5/3
        
        # avg atomic mass
        Xsol = 1.0
        Zsol = 1.0
        
        X = Xsol * 0.7381
        Z = Zsol * 0.0134
        Y = 1 - X - Z
        
        self.mu  = 1.0/(2.*X+ 3.*(1.-X-Z)/4.+ Z/2.);
        self.mue = 2.0/(1.0+X);
        self.muH = 1.0/X;
        self.mH = 1.0

        # alpha values for different sims
        self.alpha_hyd = 2 ** (1 / 3)  # 1.26
        
        self.alpha_mhd = (2 * 4.2 / 0.77) ** (1 / 3)

u = unit()

In [140]:
def calc_T(P, rho):
    """
    Calculates temeprature from constants
    ----------
    P: gas pressure
    rho: gas density
    """
    T = P/rho * u.KELVIN * u.mu
    return T

def calc_P(T, rho):
    """
    Calculates pressure from constants
    ----------
    T: gas temperature
    rho: gas density
    """
    P = (T / 64**3) * (rho / 64**3) / (u.KELVIN * u.mu)
    return P

def calc_cs(T):
    """
    Calculates sound speed
    ----------
    T: temperature
    mu: avg atomic number of the gas
    """
    # convert to cm
    m_to_cm = 100

    # return np.sqrt(g.g*R*T_hot/M) * m_to_cm/g.unit_velocity
    # return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) * m_to_cm / u.unit_velocity
    return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) / u.unit_velocity

def calc_vturb(mach, P, rho):
    """
    Calculates the turbulent velocity from desired Mach number M
    ----------
    mach: mach number
    P, rho
    """
    T = calc_T(P, rho)
    cs = calc_cs(T)
    return mach * cs
def calc_vturb(mach, T):
    """
    Calculates the turbulent velocity from desired Mach number M
    ----------
    mach: mach number
    T
    """
    cs = calc_cs(T)
    return mach * cs

def calc_mach(v_turb, P, rho):
    """
    Calculates the Mach number
    ----------
    v_turb: turbulence velocity
    P, rho
    """
    T = calc_T(P, rho)
    cs = calc_cs(T)
    print(f'cs = {cs}')
    return v_turb / cs

def calc_dedt_mach(mach, P, rho, L):
    """
    Returns required dedt for a given Mach number, pressure, density, and box size
    """
    # calculate sound speed first
    T = calc_T(P, rho)
    cs_new = calc_cs(T)

    dedt_req = rho * (cs_new**3) * (L**2) * (mach**3) / (u.alpha_hyd**3)
    return dedt_req
def calc_dedt_mach(mach, T, rho, L):
    """
    Returns required dedt for a given Mach number, temperature, and box size
    """
    # calculate sound speed first
    cs_new = calc_cs(T)

    dedt_req = rho * (cs_new**3) * (L**2) * (mach**3) / (u.alpha_hyd**3)
    return dedt_req


def calc_dedt_vturb(v_turb, rho, L):
    """
    Returns required dedt for a given turbulent velocity, density, and box size

    Does NOT calculate sound speed, so no pressure required
    """
    
    dedt_req = rho * v_turb**3 * (L**2) / (u.alpha_hyd**3)
    return dedt_req

def find_ind_l(seq, val):
    seq = list(seq)
    return seq.index(list(filter(lambda x: x>val, seq))[0])

## Input params

In [141]:
"""
Primitive parameters
"""

# control params  %% changed here
trial = '240620_0.6_320'

# cloud params
cloud_flag = True
T_cloud = 8e2  # temperature of cloud  %% changed here
cloud_radius = 8e-4   # box size is x20
cloud_pos = [0, 0, 0]

# box & mesh parameters
box_scale = 20  # how many times the cloud radius
box_size = cloud_radius * box_scale  # from -box_size/2 to +box_size/2
grid_dim = 128
grid_vol = grid_dim ** 3
grid_mesh = 32

# temperatures
T_floor = 8e2  # floor temperature %% changed here
T_cold = T_floor * 2  # tracking temperature for cold gas
T_ceil = 1e8
cloud_flag = True
cooling_flag = True

# hot gas params
T_hot = 4e6
rho_hot = 0.1  # density shouldn't be too low %% changed here
mach = float(trial.split('_')[1])  # mach number for the run %% changed here
# the restart cons vars file for turb run
# turb_cons_hdf_path = f'/freya/ptmp/mpa/wuze/data/cloud_hdfs/mach_{mach:.1f}.athdf'  # for ones that already have a mach
turb_cons_hdf_path = '../turb/Turb.cons.00100.athdf'  # for ones that need to go to turb runs

T_cut = 1e6

# turn into dict
run_params_primitive = _into_dict('trial', 'T_cut')
run_params_primitive

{'trial': '240620_0.6_320',
 'cloud_flag': True,
 'T_cloud': 800.0,
 'cloud_radius': 0.0008,
 'cloud_pos': [0, 0, 0],
 'box_scale': 20,
 'box_size': 0.016,
 'grid_dim': 128,
 'grid_vol': 2097152,
 'grid_mesh': 32,
 'T_floor': 800.0,
 'T_cold': 1600.0,
 'T_ceil': 100000000.0,
 'cooling_flag': True,
 'T_hot': 4000000.0,
 'rho_hot': 0.1,
 'mach': 0.6,
 'turb_cons_hdf_path': '../turb/Turb.cons.00100.athdf',
 'T_cut': 1000000.0}

## Derived params

In [142]:
"""
Turbulence time scales, etc.
Derived parameters
"""

# hot gas params
chi = T_hot / T_cloud
P_hot = calc_P(T_hot, rho_hot)  # scaled hot gas pressure
v_turb = calc_vturb(mach, T=T_hot)  # turbulent velocity from desired mach number
t_eddy = box_size/v_turb
cs_hot = calc_cs(T_hot)  # sound speed for hot gas

T_mix = np.sqrt(T_cloud * T_hot)
T_warm = T_mix  # set to mixed gas temperature, for tracking

# dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3 given the density & box size
dedt = calc_dedt_mach(mach, T=T_hot, rho=rho_hot, L=box_size)

# tcorr ~ t_eddy
t_corr = t_eddy

# dtdrive << t_eddy
dt_drive = t_eddy / 1e3

# the end time of simulation, multiples of time_start
# reaches >5 t_cc or t_eddy
t_cc = (chi ** (1/2) * cloud_radius / v_turb)
t_maxc = np.max([t_eddy, t_cc])  # 1 for floored, 1 for start
time_start = 10 * t_maxc  # the start time of CLOUD simulation; the end time of the TURB simulation
time_end = (((5 * t_maxc) // time_start) + 2) * time_start
dt_hdf5 = time_start / 100  # the time interval between hdf5 outputs
time_cloud = time_start  # introduce cloud when simulation starts

# turn into dict
run_params_derived = _into_dict('chi', 'time_cloud')
run_params_derived

{'chi': 5000.0,
 'P_hot': 8.438675409534824e-14,
 'v_turb': 0.1865317171670572,
 't_eddy': 0.08577629715203047,
 'cs_hot': 0.31088619527842865,
 'T_mix': 56568.5424949238,
 'T_warm': 56568.5424949238,
 'dedt': 8.307455707108616e-08,
 't_corr': 0.08577629715203047,
 'dt_drive': 8.577629715203047e-05,
 't_cc': 0.3032650069063655,
 't_maxc': 0.3032650069063655,
 'time_start': 3.0326500690636546,
 'time_end': 6.065300138127309,
 'dt_hdf5': 0.030326500690636547,
 'time_cloud': 3.0326500690636546}

## Save params

In [143]:
import os

# make datapath
datapath = f'/freya/ptmp/mpa/wuze/data/{trial}'

if not os.path.exists(datapath):
    os.makedirs(datapath)
    os.makedirs(f'{datapath}/cloud')
    os.makedirs(f'{datapath}/turb')
    print(f'Made path {datapath}')

In [144]:
# save as pickles
import pickle

run_params_all = {**run_params_primitive, **run_params_derived}

with open(f'{datapath}/params.pickle', 'wb') as handle:
    pickle.dump(run_params_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

run_params_all

{'trial': '240620_0.6_320',
 'cloud_flag': True,
 'T_cloud': 800.0,
 'cloud_radius': 0.0008,
 'cloud_pos': [0, 0, 0],
 'box_scale': 20,
 'box_size': 0.016,
 'grid_dim': 128,
 'grid_vol': 2097152,
 'grid_mesh': 32,
 'T_floor': 800.0,
 'T_cold': 1600.0,
 'T_ceil': 100000000.0,
 'cooling_flag': True,
 'T_hot': 4000000.0,
 'rho_hot': 0.1,
 'mach': 0.6,
 'turb_cons_hdf_path': '../turb/Turb.cons.00100.athdf',
 'T_cut': 1000000.0,
 'chi': 5000.0,
 'P_hot': 8.438675409534824e-14,
 'v_turb': 0.1865317171670572,
 't_eddy': 0.08577629715203047,
 'cs_hot': 0.31088619527842865,
 'T_mix': 56568.5424949238,
 'T_warm': 56568.5424949238,
 'dedt': 8.307455707108616e-08,
 't_corr': 0.08577629715203047,
 'dt_drive': 8.577629715203047e-05,
 't_cc': 0.3032650069063655,
 't_maxc': 0.3032650069063655,
 'time_start': 3.0326500690636546,
 'time_end': 6.065300138127309,
 'dt_hdf5': 0.030326500690636547,
 'time_cloud': 3.0326500690636546}

# Set params

In [145]:
"""Define run parameters"""

class run_params():
    def __init__(self, run_params_all):
        self.trial = run_params_all['trial']
        self.cloud_flag = run_params_all['cloud_flag']
        self.T_cloud = run_params_all['T_cloud']
        self.T_cold = run_params_all['T_cold']
        self.cloud_radius = run_params_all['cloud_radius']
        self.cloud_pos = run_params_all['cloud_pos']
        self.box_size = run_params_all['box_size']
        self.grid_dim = run_params_all['grid_dim']
        self.grid_vol = run_params_all['grid_vol']
        self.grid_mesh = run_params_all['grid_mesh']
        self.T_floor = run_params_all['T_floor']
        self.T_ceil = run_params_all['T_ceil']
        self.cooling_flag = run_params_all['cooling_flag']
        self.T_hot = run_params_all['T_hot']
        self.rho_hot = run_params_all['rho_hot']
        self.mach = run_params_all['mach']
        self.T_cut = run_params_all['T_cut']
        self.chi = run_params_all['chi']
        self.P_hot = run_params_all['P_hot']
        self.v_turb = run_params_all['v_turb']
        self.t_eddy = run_params_all['t_eddy']
        self.cs_hot = run_params_all['cs_hot']
        self.T_mix = run_params_all['T_mix']
        self.T_warm = run_params_all['T_warm']
        self.dedt = run_params_all['dedt']
        self.t_corr = run_params_all['t_corr']
        self.dt_drive = run_params_all['dt_drive']
        self.t_cc = run_params_all['t_cc']
        self.t_maxc = run_params_all['t_maxc']
        self.time_start = run_params_all['time_start']
        self.time_end = run_params_all['time_end']
        self.dt_hdf5 = run_params_all['dt_hdf5']
        self.time_cloud = run_params_all['time_cloud']

rp = run_params(run_params_all)
rp.trial

'240620_0.6_320'

# Gen .turb

## Readme

In [146]:
"""
Generate readme
"""

def infgen_readme(datapath, readme_msg="This run"):
    f = open(f"{datapath}/readme", "w")
    
    # generate text
    readme_text =\
    f"""
{readme_msg}

{'-' * 50}
This run is auto-generated by the trial_gen.ipynb

This run uses:
    """
    
    # generate variables
    readme_vars = f'\nPrimitive\n{'-' * 20}\n'
    for key, val in run_params_primitive.items():
        if isinstance(val, float):
            app = f'{key} = {val:.4f}\n'
        else:
            app = f'{key} = {val}\n'
        readme_vars += app
    readme_vars += f'\nDerived\n{'-' * 20}\n'
    for key, val in run_params_derived.items():
        if isinstance(val, float):
            app = f'{key} = {val:.4f}\n'
        else:
            app = f'{key} = {val}\n'
        readme_vars += app
    
    f.write(readme_text)
    f.write('-' * 50)
    f.write(readme_vars)
    f.close()

## athinput_init.turb

In [147]:
"""
Generate TURB .turb file
"""
def infgen_init_turb(datapath):
    f = open(f"{datapath}/turb/athinput_init.turb", "w")
    
    # generate text
    turb_config =\
    f""" 
<comment>
problem   = Adding turbulence
configure = --prob=turb_v2 -fft 

<job>
problem_id = Turb # problem ID: basename of output filenames


#~----------OUTPUTS-----------~#
<output1>
file_type  = hst        # history data dump
dt         = {dt_hdf5 / 100 :<20.10f}# time increment between outputs

<output2>
file_type = hdf5        # HDF5 data dump
variable  = prim        # variables to be output
dt        = {dt_hdf5:<20.10f}# time increment between outputs

<output3>
file_type = rst         # restart file dump                       
dt        = {dt_hdf5 * 10 :<20.10f}# time increment between outputs

<output4>
file_type = hdf5        # HDF5 data dump
variable  = cons        # variables to be output
dt        = {dt_hdf5:<20.10f}# time increment between outputs
id        = cons

#~----------SETUP-----------~#
<time>
cfl_number  = 0.3       # The Courant, Friedrichs, & Lewy (CFL) Number
nlim        = -1        # cycle limit
# time limit of second trial should be larger
tlim        = {time_start:<20.10f}# time limit for the turb run, where the cloud run starts
integrator  = rk2       # time integration algorithm
xorder      = 2         # order of spatial reconstruction
ncycle_out  = 1         # interval for stdout summary info

<mesh>
nx1    = {grid_dim}             # number of zones in x1-direction
x1min  = {-box_size/2}           # minimum value of x1
x1max  = {box_size/2}            # maximum value of x1
ix1_bc = periodic       # inner-x1 boundary condition
ox1_bc = periodic       # outer-x1 boundary condition

nx2    = {grid_dim}             # number of zones in x2-direction
x2min  = {-box_size/2}           # minimum value of x2
x2max  = {box_size/2}            # maximum value of x2
ix2_bc = periodic       # inner-x2 boundary condition
ox2_bc = periodic       # outer-x2 boundary condition

nx3    = {grid_dim}             # number of zones in x3-direction
x3min  = {-box_size/2}           # minimum value of x3
x3max  = {box_size/2}            # maximum value of x3
ix3_bc = periodic       # inner-x3 boundary condition
ox3_bc = periodic       # outer-x3 boundary condition

refinement  = none      # type of mesh refinement to use

<meshblock>
nx1 = {grid_mesh}                # block size in x1-direction
nx2 = {grid_mesh}                # block size in x2-direction
nx3 = {grid_mesh}                # block size in x3-direction

<hydro>
gamma      = 1.6666666666666667  # gamma = C_p/C_v
pfloor     = 0.0005              # pressure floor

#~----------TURBULENCE PARAMS-----------~#
<turbulence>
dedt       = {dedt:<20.15f}# Energy injection rate (for driven) or Total energy (for decaying)
nlow       = 0                   # cut-off wavenumber at low-k
nhigh      = 2                   # cut-off wavenumber at high-k
expo       = 2.0                 # power-law exponent
tcorr      = {t_corr:<20.15f}# correlation time for OU process (both impulsive and continuous)
dtdrive    = {dt_drive:<20.15f}# time interval between perturbation (impulsive)
f_shear    = 0.3                 # the ratio of the shear component
rseed      = 1                   # if non-negative, seed will be set by hand (slow PS generation)
# dedt should be calibrated by        dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3
# tcorr ~ t_eddy
# dtdrive << t_eddy

<problem>
turb_flag    = 2                 # 1 for decaying, 2 (impulsive) or 3 (continuous) for driven turbulence
rescale_flag = 1                 # 1 for cloud runs


#~----------HEATING & COOLING-----------~#
# User-defined variables:

heating = 0.001                  # constant volume heating rate

cs_hot = {cs_hot:<20.10f}# hot gas sound speed for T~4e6K

# turn on cooling for cloud runs
cooling_flag = 0                 # set to 1 to turn on cooling, 0 to turn it off
global_cooling_flag = 0          # set to 1 to turn on uniform global cooling, 0 to turn it off
# turn on cloud
restart_cloud_flag   = 0                 # set to 1 to add the cloud on restart, 0 to not 
hdf_cloud_flag       = 0                 # set to 1 to add the cloud after reading HDF5 file, 0 to not 

amb_rho      = {rho_hot}               # density of the ambient medium, in code units


#~----------CLOUD PROPERTIES-----------~#
cloud_radius = {cloud_radius:<20.10f}# radius of the cloud, in code units
# this should be the same as simulation start
start_time   = {time_start:<20.10f}# time of insertion of cloud, in code units
cloud_time   = {time_cloud:<20.10f}# time of insertion of cloud, in code units
# temperature ratio T_init / T_cloud
# TO SET COLD GAS TEMPERATURE, CHANGE HERE
cloud_chi    = {chi}               # density contrast of the cloud, rho_cloud/amb_rho

cloud_pos_x  = {cloud_pos[0]}               # cloud center position x-coordinate
cloud_pos_y  = {cloud_pos[1]}               # cloud center position y-coordinate
cloud_pos_z  = {cloud_pos[2]}               # cloud center position z-coordinate

#~-----------TEMPERATURE----------~#
T_floor      = {T_floor:<10.0f}# floor temperature in the simulation
T_ceil       = {T_ceil:<10.0f}# ceiling temperature in the simulation

# medium mass is integrated from 2 x temp
T_hot_req    = {T_hot:<10.0f}# hot medium temperature required, reset to this on restart, if cloud_flag is 1
T_hot        = {T_hot:<10.0f}# initial hot medium temperature (box heats up due to turbulence)
T_cold       = {T_cold:<10.0f}# cold medium temperature, ONLY used in cold gas mass calculation
T_cut_mul    = 0.5               # T_cut = T_hot_req * T_cut_mul, gas higher than T_cut is not cooled
# infinite cooling time for T > T_cut, want hot gas to stay hot for the medium
T_cut        = {T_cut:<10.0f}# gas higher than T_cut is not cooled
T_warm       = {T_warm:<10.2f}# warm medium temperature, ONLY used in warm gas mass calculation

Xsol         = 1.0               # Change H-abundance src/utils/code_units
Zsol         = 1.0               # Change metallicity src/utils/code_units

B_x          = 0.0               # initial magnetic field in x-direction
B_y          = 0.0               # initial magnetic field in y-direction
B_z          = 0.0               # initial magnetic field in z-direction

# only read if hdf_cloud_flag = 1
cons_input_filename = /dev/null  # name of HDF5 file containing initial conditions
dataset_cons   = cons       # name of dataset containing conserved values
index_dens     = 0          # index of density in conserved dataset
index_etot     = 1          # index of energy in conserved dataset (for nonbarotropic EOS)
index_mom1     = 2          # index of x1-momentum in conserved dataset
index_mom2     = 3          # index of x2-momentum in conserved dataset
index_mom3     = 4          # index of x3-momentum in conserved dataset

# only used if -b flag is on for compiling
b1_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b2_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b3_input_filename = /dev/null  # name of HDF5 file containing initial conditions
    """
    
    # generate variables
    f.write(turb_config)
    f.write('-' * 50)
    f.close()

## athinput_cloud.turb

In [148]:
"""
Generate CLOUD .turb file
"""
def infgen_cloud_turb(datapath):
    f = open(f"{datapath}/cloud/athinput_cloud.turb", "w")
    
    # generate text
    cloud_config =\
    f"""
<comment>
problem   = Adding cloud
configure = --prob=turb_v2 -fft 

<job>
problem_id = Turb # problem ID: basename of output filenames


#~----------OUTPUTS-----------~#
<output1>
file_type  = hst        # history data dump
dt         = {dt_hdf5 / 100 :<20.10f}# time increment between outputs

<output2>
file_type = hdf5        # HDF5 data dump
variable  = prim        # variables to be output
dt        = {dt_hdf5:<20.10f}# time increment between outputs

<output3>
file_type = rst         # restart file dump                       
dt        = {dt_hdf5 * 10 :<20.10f}# time increment between outputs

<output4>
file_type = hdf5        # HDF5 data dump
variable  = cons        # variables to be output
dt        = {dt_hdf5:<20.10f}# time increment between outputs
id        = cons

#~----------SETUP-----------~#
<time>
cfl_number  = 0.3       # The Courant, Friedrichs, & Lewy (CFL) Number
nlim        = -1        # cycle limit
# time limit of second trial should be larger
tlim        = {time_end:<20.10f}# time limit
integrator  = rk2       # time integration algorithm
xorder      = 2         # order of spatial reconstruction
ncycle_out  = 1         # interval for stdout summary info

<mesh>
nx1    = {grid_dim}             # number of zones in x1-direction
x1min  = {-box_size/2}           # minimum value of x1
x1max  = {box_size/2}            # maximum value of x1
ix1_bc = periodic       # inner-x1 boundary condition
ox1_bc = periodic       # outer-x1 boundary condition

nx2    = {grid_dim}             # number of zones in x2-direction
x2min  = {-box_size/2}           # minimum value of x2
x2max  = {box_size/2}            # maximum value of x2
ix2_bc = periodic       # inner-x2 boundary condition
ox2_bc = periodic       # outer-x2 boundary condition

nx3    = {grid_dim}             # number of zones in x3-direction
x3min  = {-box_size/2}           # minimum value of x3
x3max  = {box_size/2}            # maximum value of x3
ix3_bc = periodic       # inner-x3 boundary condition
ox3_bc = periodic       # outer-x3 boundary condition

refinement  = none      # type of mesh refinement to use

<meshblock>
nx1 = {grid_mesh}                # block size in x1-direction
nx2 = {grid_mesh}                # block size in x2-direction
nx3 = {grid_mesh}                # block size in x3-direction

<hydro>
gamma      = 1.6666666666666667  # gamma = C_p/C_v
pfloor     = 0.0005              # pressure floor

#~----------TURBULENCE PARAMS-----------~#
<turbulence>
dedt       = {dedt:<20.15f}# Energy injection rate (for driven) or Total energy (for decaying)
nlow       = 0                   # cut-off wavenumber at low-k
nhigh      = 2                   # cut-off wavenumber at high-k
expo       = 2.0                 # power-law exponent
tcorr      = {t_corr:<20.15f}# correlation time for OU process (both impulsive and continuous)
dtdrive    = {dt_drive:<20.15f}# time interval between perturbation (impulsive)
f_shear    = 0.3                 # the ratio of the shear component
rseed      = 1                   # if non-negative, seed will be set by hand (slow PS generation)
# dedt should be calibrated by        dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3
# tcorr ~ t_eddy
# dtdrive << t_eddy

<problem>
turb_flag    = 2                 # 1 for decaying, 2 (impulsive) or 3 (continuous) for driven turbulence
rescale_flag = 1                 # 1 for cloud runs


#~----------HEATING & COOLING-----------~#
# User-defined variables:

heating = 0.001                  # constant volume heating rate

cs_hot = {cs_hot:<20.10f}# hot gas sound speed for T~4e6K

# turn on cooling for cloud runs
cooling_flag = {1 if cooling_flag else 0}                 # set to 1 to turn on cooling, 0 to turn it off
global_cooling_flag = 0          # set to 1 to turn on uniform global cooling, 0 to turn it off
# turn on cloud
restart_cloud_flag   = 0                 # set to 1 to add the cloud on restart, 0 to not 
hdf_cloud_flag       = {1 if cloud_flag else 0}                 # set to 1 to add the cloud after reading HDF5 file, 0 to not 

amb_rho      = {rho_hot}               # density of the ambient medium, in code units


#~----------CLOUD PROPERTIES-----------~#
cloud_radius = {cloud_radius:<20.10f}# radius of the cloud, in code units
# this should be the same as simulation start
start_time   = {time_start:<20.10f}# time of insertion of cloud, in code units
cloud_time   = {time_cloud:<20.10f}# time of insertion of cloud, in code units
# temperature ratio T_init / T_cloud
# TO SET COLD GAS TEMPERATURE, CHANGE HERE
cloud_chi    = {chi}               # density contrast of the cloud, rho_cloud/amb_rho

cloud_pos_x  = {cloud_pos[0]}               # cloud center position x-coordinate
cloud_pos_y  = {cloud_pos[1]}               # cloud center position y-coordinate
cloud_pos_z  = {cloud_pos[2]}               # cloud center position z-coordinate

#~-----------TEMPERATURE----------~#
T_floor      = {T_floor:<10.0f}# floor temperature in the simulation
T_ceil       = {T_ceil:<10.0f}# ceiling temperature in the simulation

# medium mass is integrated from 2 x temp
T_hot_req    = {T_hot:<10.0f}# hot medium temperature required, reset to this on restart, if cloud_flag is 1
T_hot        = {T_hot:<10.0f}# initial hot medium temperature (box heats up due to turbulence)
T_cold       = {T_cold:<10.0f}# cold medium temperature, ONLY used in cold gas mass calculation
T_cut_mul    = 0.5               # T_cut = T_hot_req * T_cut_mul, gas higher than T_cut is not cooled
# infinite cooling time for T > T_cut, want hot gas to stay hot for the medium
T_cut        = {T_cut:<10.0f}# gas higher than T_cut is not cooled
T_warm       = {T_warm:<10.2f}# warm medium temperature, ONLY used in warm gas mass calculation

Xsol         = 1.0               # Change H-abundance src/utils/code_units
Zsol         = 1.0               # Change metallicity src/utils/code_units

B_x          = 0.0               # initial magnetic field in x-direction
B_y          = 0.0               # initial magnetic field in y-direction
B_z          = 0.0               # initial magnetic field in z-direction

# only read if hdf_cloud_flag = 1
cons_input_filename = {turb_cons_hdf_path}  # name of HDF5 file containing initial conditions
dataset_cons   = cons       # name of dataset containing conserved values
index_dens     = 0          # index of density in conserved dataset
index_etot     = 1          # index of energy in conserved dataset (for nonbarotropic EOS)
index_mom1     = 2          # index of x1-momentum in conserved dataset
index_mom2     = 3          # index of x2-momentum in conserved dataset
index_mom3     = 4          # index of x3-momentum in conserved dataset

# only used if -b flag is on for compiling
b1_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b2_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b3_input_filename = /dev/null  # name of HDF5 file containing initial conditions
    """
    
    # generate variables
    f.write(cloud_config)
    f.write('-' * 50)
    f.close()

## Write files

In [149]:
# readme
readme_msg = """
This trial keeps the gas starting out at the temperature floor of 800
"""
infgen_readme(datapath=datapath, readme_msg=readme_msg)

In [150]:
# init / turb
infgen_init_turb(datapath=datapath)

In [151]:
# cloud
infgen_cloud_turb(datapath=datapath)

In [152]:
import datetime
print(datetime.datetime.now())
print('\n' * 10)

2024-06-20 15:37:36.847654













In [153]:
pass

# Gen .sh

In [154]:
"""
Get trial params
"""
import datetime
date = str(datetime.datetime.now().date()).replace('-', '')[2:]  # six digit date
run_dp = trial[7:]  # x_y

"""
Machine params
"""
executable = 'athena_turb'
server = '24h' #test
if server == '24h':
    comp_time = '04:00:00'
    comp_time_save = '03:55:00'
elif server == 'test':
    comp_time = '00:20:00'
    comp_time_save = '00:19:00'

print(date, run_dp, executable, server, comp_time, comp_time_save)

240620 0.6_320 athena_turb 24h 04:00:00 03:55:00


In [155]:
sh_content = f"""#!/usr/bin/env bash
# source ~/.bashrc

#SBATCH -J {date[2:]}{trial[6:]}_turb
#SBATCH -o ./out/{date}/{run_dp}_turb."%j".out
#SBATCH -e ./out/{date}/{run_dp}_turb."%j".err
#SBATCH --mail-user wuz@mpa-garching.mpg.de
#-- wuz@mpa-garching.mpg.de
#SBATCH --partition=p.24h
#-- p.24h, p.test, p.gpu & p.gpu.ampere
#SBATCH --mail-type=ALL
#SBATCH --nodes=2
#-- 2 at most
#SBATCH --ntasks-per-node=32
#-- 40 at most
#SBATCH --time={comp_time}
#-- in format hh:mm:ss

set -e
SECONDS=0

module purge
module load intel/19.1.2
module load impi/2019.8
module load fftw-mpi/3.3.8
module load hdf5-mpi/1.8.21
module load ffmpeg/4.4
module list

echo {trial}
cd $DP/{trial}/turb

srun ../../{executable} -i athinput_init.turb -t {comp_time_save}
#-- !!TIME_LIMIT_RST!! should be ten minutes before the end, to generate a restart file

echo "Elapsed: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"

echo "Boom!"

cd ~
"""

# write the file
file_name = f"{datapath}/js_turb.sh"
with open(file_name, 'w') as file:
    file.write(sh_content)

# print the squeue command
squeue_cmd = f"""
sbatch {datapath}/js_turb.sh
"""
print(squeue_cmd)


sbatch /freya/ptmp/mpa/wuze/data/240620_0.6_320/js_turb.sh



In [156]:
sh_content = f"""#!/usr/bin/env bash
# source ~/.bashrc

#SBATCH -J {date[2:]}{trial[6:]}_cloud
#SBATCH -o ./out/{date}/{run_dp}_cloud."%j".out
#SBATCH -e ./out/{date}/{run_dp}_cloud."%j".err
#SBATCH --mail-user wuz@mpa-garching.mpg.de
#-- wuz@mpa-garching.mpg.de
#SBATCH --partition=p.24h
#-- p.24h, p.test, p.gpu & p.gpu.ampere
#SBATCH --mail-type=ALL
#SBATCH --nodes=2
#-- 2 at most
#SBATCH --ntasks-per-node=32
#-- 40 at most
#SBATCH --time={comp_time}
#-- in format hh:mm:ss

set -e
SECONDS=0

module purge
module load intel/19.1.2
module load impi/2019.8
module load fftw-mpi/3.3.8
module load hdf5-mpi/1.8.21
module load ffmpeg/4.4
module list

echo {trial}
cd $DP/{trial}/cloud

srun ../../{executable} -i athinput_cloud.turb -t {comp_time_save}
#-- !!TIME_LIMIT_RST!! should be ten minutes before the end, to generate a restart file

echo "Elapsed: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"

echo "Boom!"

cd ~
"""

# write the file
file_name = f"{datapath}/js_cloud.sh"
with open(file_name, 'w') as file:
    file.write(sh_content)

# print the squeue command
squeue_cmd = f"""
sbatch {datapath}/js_cloud.sh
"""
print(squeue_cmd)


sbatch /freya/ptmp/mpa/wuze/data/240620_0.6_320/js_cloud.sh



In [157]:
!mkdir -p /freya/ptmp/mpa/wuze/out/{date}

In [158]:
pass